In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
import matplotlib.pyplot as plt
import math
import soundfile as sf
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, mixed_precision
from tensorflow.keras.utils import plot_model
from sklearn import preprocessing
import sys
from keras import backend as K
from sklearn.model_selection import train_test_split, StratifiedKFold
import matplotlib.pyplot as plt
import json
import gc

df = pd.DataFrame(np.load('../datasets/gtzan10sAug/Final/features/a0.5_min1.0_max10.0_3.0s_block_0.5s_hop.npy', allow_pickle=True), columns=['melspec','label','filename'])
X = np.stack(df[['melspec','filename']].values)
y = df['label'].to_numpy()
names = df['filename'].to_numpy()
del df
gc.collect()
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

{  

    'blues': 0,  
    'classical': 1,  
    'country': 2,  
    'disco': 3,  
    'hiphop': 4,  
    'jazz': 5,  
    'metal': 6,  
    'pop': 7,  
    'reggae': 8,  
    'rock': 9   
    
}

In [ ]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.30, shuffle=True, random_state=42)

In [ ]:
x_test, x_val, y_test, y_val = train_test_split(x_val, y_val, test_size=0.50, shuffle=True, random_state=42)

In [ ]:
np.save('x_train.npy', x_train)
np.save('x_test.npy',x_test)
np.save('x_val.npy',x_val)
np.save('y_train.npy', y_train)
np.save('y_test.npy',y_test)
np.save('y_val.npy', y_val)


In [ ]:
df=pd.DataFrame()
df['melspec'] = list(x_train[:,0])
df['label'] = y_train
df['filename'] = x_train[:,1]
df

In [ ]:
for idx, row in tqdm(df.iterrows()):
    filename = row['filename']
    np.save(f'../datasets/gtzan10sAug/vedant/train/{filename}.npy',row[['melspec','label']])

In [ ]:
import os
_,_,filenames = next(os.walk('../datasets/gtzan10sAug/vedant/train'))
print(len(filenames))

In [ ]:
class AugmentedDataset(keras.utils.Sequence):
    def __init__(self, mode, batch_size):
        self.batch_size = batch_size
        self.mode = mode
        if self.mode == 'train':
            self.data_path = '../datasets/gtzan10sAug/vedant/train'
        if self.mode == 'test':
            self.data_path = '../datasets/gtzan10sAug/vedant/test'
        if self.mode == 'val':
            self.data_path = '../datasets/gtzan10sAug/vedant/val'
        _,_,self.filenames = next(os.walk(self.data_path))
    def __len__(self):
        return len(self.filenames)

    def __getitem__(self,idx):
        batch = self.filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        bat = np.array([np.load(os.path.join(self.data_path, x), allow_pickle=True) for x in batch])
        return np.array(bat[:,0]), np.array(bat[:,1])

In [ ]:
batch_size = 32
train_dataset = AugmentedDataset('train', batch_size)
test_dataset = AugmentedDataset('test', batch_size)
val_dataset = AugmentedDataset('val', batch_size)

In [ ]:
a = next(iter(test_dataset))
a

## Example usage:

In [ ]:
model.fit_generator(generator=train_dataset,
                   steps_per_epoch = int(len(train_dataset) // batch_size),
                   epochs = 10,
                   verbose = 1,
                   validation_data = val_dataset,
                   validation_steps = int(len(val_dataset) // batch_size))